In [54]:
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# %%capture
from transformers import AutoTokenizer, AutoModel, pipeline, OPTForCausalLM

model_name="facebook/opt-350m"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_name, output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model=model_name)
#generator = pipeline('text-generation', model=model)


In [13]:
# for name, param in model.named_parameters():
for name, param in model.named_parameters():
    print(name)

model.decoder.embed_tokens.weight
model.decoder.embed_positions.weight
model.decoder.project_out.weight
model.decoder.project_in.weight
model.decoder.layers.0.self_attn.k_proj.weight
model.decoder.layers.0.self_attn.k_proj.bias
model.decoder.layers.0.self_attn.v_proj.weight
model.decoder.layers.0.self_attn.v_proj.bias
model.decoder.layers.0.self_attn.q_proj.weight
model.decoder.layers.0.self_attn.q_proj.bias
model.decoder.layers.0.self_attn.out_proj.weight
model.decoder.layers.0.self_attn.out_proj.bias
model.decoder.layers.0.self_attn_layer_norm.weight
model.decoder.layers.0.self_attn_layer_norm.bias
model.decoder.layers.0.fc1.weight
model.decoder.layers.0.fc1.bias
model.decoder.layers.0.fc2.weight
model.decoder.layers.0.fc2.bias
model.decoder.layers.0.final_layer_norm.weight
model.decoder.layers.0.final_layer_norm.bias
model.decoder.layers.1.self_attn.k_proj.weight
model.decoder.layers.1.self_attn.k_proj.bias
model.decoder.layers.1.self_attn.v_proj.weight
model.decoder.layers.1.self_a

In [55]:
import torch.nn.utils.prune 
def get_module_name(param_name):
    # print(param_name)
    # print(param_name[-7:])
    # print(param_name[-5:])
    if param_name[-5:] == ".bias":
        return param_name[:-5], "bias"
    elif param_name[-7:] == ".weight":
        return param_name[:-7], "weight"
    else:
        return None, None

# mask the whole model
def mask(model, amount=.2):
    module_dict = {}
    for n, m in model.named_modules():
        module_dict[n] = m
    # print(module_dict.keys())
    
    parameter_list = []
    for n, m in model.named_parameters():
        parameter_list.append(n)
    # print(parameter_list)

    for n in parameter_list:
        module_name, param_type = get_module_name(n)
        if module_name is None or param_type is None or param_type=="bias":
            continue
        # perform the masking
        module = module_dict[module_name]
        torch.nn.utils.prune.l1_unstructured(module=module, name=param_type, amount=.2)

def test_output(inp_model, inp_tokenizer, str_input):
    # input1 = tokenizer("Hello, my dog is cute.", return_tensors="pt", padding="max_length", truncation=True)
    input2 = inp_tokenizer(str_input, return_tensors="pt", padding="max_length", truncation=True)
    output = inp_model.generate(input2.input_ids, max_length=100, num_return_sequences=1, temperature=0.5, top_p=0.95)
    return inp_tokenizer.decode(output[0], skip_special_tokens=True)

In [58]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_name, output_attentions=True, output_hidden_states=True)

test_str = "What the fuck did you say to me, you bitch?"
# print("Before masking: ")
# print(test_output(model, tokenizer, test_str))
mask(model, amount=.1)
# print("after masking:")
# print(test_output(model, tokenizer, test_str))
for name, param in model.named_parameters():
    print(name)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Before masking: 
What the fuck did you say to me, you bitch?                                                                                       
after masking:
What the fuck did you say to me, you bitch?  out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out out


In [38]:
from torch.nn.utils import prune
from torch import nn 
test_tensor = nn.Linear(100, 200),
m = prune.l1_unstructured(test_tensor, 'weight', amount=0.2)
print(test_tensor)
print(m)

AttributeError: 'tuple' object has no attribute '_forward_pre_hooks'

In [49]:

m = prune.l1_unstructured(nn.Linear(8, 30), 'weight', amount=0.2)
m.state_dict().keys()
for name, param in m.named_parameters():
    print(name)
    # print(torch.count_nonzero(param))
    print(param)
print(torch.count_nonzero(m.weight))
print(m.weight)


bias
Parameter containing:
tensor([ 0.1171,  0.1990,  0.3202,  0.1351, -0.1237, -0.1750, -0.0212, -0.1523,
         0.2216,  0.1811, -0.2169, -0.0760,  0.3388,  0.0307,  0.3062,  0.2633,
        -0.2996, -0.2629, -0.3358, -0.1042, -0.0469,  0.0290, -0.3214,  0.3023,
         0.0102, -0.0627,  0.2859,  0.2337,  0.2483,  0.0767],
       requires_grad=True)
weight_orig
Parameter containing:
tensor([[-0.1567,  0.0963,  0.2458,  0.3305,  0.0441,  0.2786, -0.1870, -0.1957],
        [ 0.0016, -0.1178, -0.1468,  0.2908,  0.0982, -0.2469,  0.3241,  0.0692],
        [ 0.0327,  0.1156,  0.0157,  0.3068, -0.2122, -0.2257,  0.3365,  0.2949],
        [-0.3061, -0.2291, -0.1134, -0.3054,  0.3363,  0.0986, -0.1247, -0.1467],
        [ 0.0854, -0.2776,  0.1050, -0.0131, -0.1928, -0.2517, -0.1844,  0.2568],
        [ 0.2989,  0.0898, -0.2867, -0.3061, -0.3236, -0.2317,  0.1141, -0.1944],
        [-0.1580, -0.3144,  0.1055, -0.2323, -0.1740, -0.0539, -0.3429,  0.2824],
        [ 0.1459, -0.0139, -0.2712,